# Imports

In [1]:
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
from imutils import face_utils
import os
import time
from IPython.display import display, Image
from tqdm.notebook import tqdm

# Configuration de matplotlib
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 5)

print("Bibliothèques importées avec succès !")

Bibliothèques importées avec succès !


# Paramètres d'enregistrement


In [3]:
frame_rate = 30  # Images par seconde
duration = 1     # Durée d'enregistrement en secondes (1 seconde suffit pour les mots courts)
width = 640      # Largeur de la vidéo
height = 480     # Hauteur de la vidéo

# Mots cibles à enregistrer
target_words = ["oui", "non", "un", "deux"]

# Nombre d'exemples par mot
examples_per_word = 10

# Chemins pour sauvegarder les fichiers
video_dir = "../data/video_recordings"
lip_dir = "../data/lip_frames"
# Vérifier que les dossiers existent
for word in target_words:
    word_video_dir = os.path.join(video_dir, word)
    word_lip_dir = os.path.join(lip_dir, word)
    os.makedirs(word_video_dir, exist_ok=True)
    os.makedirs(word_lip_dir, exist_ok=True)
    print(f"Dossiers créés : {word_video_dir} et {word_lip_dir}")

print("\nConfiguration terminée !")

Dossiers créés : ../data/video_recordings\oui et ../data/lip_frames\oui
Dossiers créés : ../data/video_recordings\non et ../data/lip_frames\non
Dossiers créés : ../data/video_recordings\un et ../data/lip_frames\un
Dossiers créés : ../data/video_recordings\deux et ../data/lip_frames\deux

Configuration terminée !


# Initialisation du détecteur de visage et du prédicteur de points de repère


In [4]:
detector = dlib.get_frontal_face_detector()
predictor_path = "../models/shape_predictor_68_face_landmarks.dat"
if os.path.exists(predictor_path):
    predictor = dlib.shape_predictor(predictor_path)
    print("Détecteur de points de repère chargé !")
else:
    print(f"ATTENTION: Le fichier {predictor_path} n'a pas été trouvé.")
    print("Téléchargez-le depuis http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
    print("Puis décompressez-le et placez-le dans le dossier 'models'")


Détecteur de points de repère chargé !


In [5]:
def record_video(duration=duration, frame_rate=frame_rate, width=width, height=height):
    """
    Enregistre une vidéo avec la webcam et retourne la liste des images.
    """
    print(f"Enregistrement de {duration} secondes...")
    
    # Initialisation de la webcam
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    
    # Countdown
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(0.5)
        
    print("Parlez maintenant !")
    
    frames = []
    start_time = time.time()
    
    while time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convertir BGR en RGB pour l'affichage avec matplotlib
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame_rgb)
        
        # Afficher l'image en temps réel
        cv2.imshow('Recording', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Libérer la webcam et fermer les fenêtres
    cap.release()
    cv2.destroyAllWindows()
    
    print("Enregistrement terminé !")
    return frames

In [6]:
def extract_lip_region(frame):
    """
    Extrait la région des lèvres d'une image.
    """
    # Convertir en niveaux de gris pour la détection
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Détecter les visages
    faces = detector(gray, 0)
    
    if len(faces) == 0:
        return None
    
    # Prendre le premier visage détecté
    face = faces[0]
    
    # Prédire les points de repère
    shape = predictor(gray, face)
    shape = face_utils.shape_to_np(shape)
    
    # Les points des lèvres sont les points 48-68 dans le modèle à 68 points
    lips_points = shape[48:68]
    
    # Calculer le rectangle englobant pour les lèvres
    x, y = lips_points.min(axis=0)
    w, h = lips_points.max(axis=0) - lips_points.min(axis=0)
    
    # Ajouter une marge
    margin = 10
    x = max(0, x - margin)
    y = max(0, y - margin)
    w = min(frame.shape[1] - x, w + 2 * margin)
    h = min(frame.shape[0] - y, h + 2 * margin)
    
    # Extraire la région des lèvres
    lip_region = frame[y:y+h, x:x+w]
    
    return lip_region

In [7]:
def visualize_recording(frames):
    """
    Affiche un aperçu de l'enregistrement et des régions des lèvres extraites.
    """
    if not frames:
        print("Aucune image à afficher !")
        return
    
    # Extraire les régions des lèvres
    lip_regions = []
    for frame in frames:
        lip_region = extract_lip_region(frame)
        if lip_region is not None:
            lip_regions.append(lip_region)
    
    # Sélectionner quelques images à afficher
    num_display = min(4, len(frames))
    if num_display > 0:
        display_indices = np.linspace(0, len(frames)-1, num_display, dtype=int)
        
        plt.figure(figsize=(16, 8))
        
        for i, idx in enumerate(display_indices):
            frame = frames[idx]
            
            # Afficher l'image complète
            plt.subplot(2, num_display, i+1)
            plt.imshow(frame)
            plt.title(f'Image {idx+1}')
            plt.axis('off')
            
            # Afficher la région des lèvres si disponible
            plt.subplot(2, num_display, i+1+num_display)
            if idx < len(lip_regions):
                lip_region = lip_regions[idx]
                plt.imshow(lip_region)
                plt.title(f'Lèvres {idx+1}')
            else:
                plt.text(0.5, 0.5, 'Lèvres non détectées', ha='center', va='center')
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Afficher les statistiques
    if lip_regions:
        print(f"Nombre d'images avec lèvres détectées : {len(lip_regions)}/{len(frames)}")
        print(f"Taille moyenne de la région des lèvres : {np.mean([r.shape[0] for r in lip_regions]):.1f} x {np.mean([r.shape[1] for r in lip_regions]):.1f} pixels")
    else:
        print("Aucune lèvre détectée dans les images!")
    
    return lip_regions

In [10]:
def save_video_and_lips(frames, word, example_index):
    """
    Sauvegarde la vidéo et les régions des lèvres.
    """
    if not frames:
        print("Aucune image à sauvegarder !")
        return [], []
    
    # Chemins pour la vidéo
    video_filename = f"{word}_{example_index:02d}.mp4"
    video_path = os.path.join(video_dir, word, video_filename)
    
    # Taille de la première image
    h, w = frames[0].shape[:2]
    
    # Créer l'objet VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_path, fourcc, frame_rate, (w, h))
    
    # Extraire et sauvegarder les régions des lèvres
    lip_paths = []
    
    for i, frame in enumerate(frames):
        # Convertir RGB en BGR pour OpenCV
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        out.write(frame_bgr)
        
        # Extraire et sauvegarder la région des lèvres
        lip_region = extract_lip_region(frame)
        if lip_region is not None:
            lip_filename = f"{word}_{example_index:02d}_{i+1:03d}.png"
            lip_path = os.path.join(lip_dir, word, lip_filename)
            cv2.imwrite(lip_path, cv2.cvtColor(lip_region, cv2.COLOR_RGB2BGR))
            lip_paths.append(lip_path)
    
    # Libérer les ressources
    out.release()
    
    print(f"Vidéo sauvegardée : {video_path}")
    print(f"{len(lip_paths)} images de lèvres sauvegardées dans {os.path.join(lip_dir, word)}")
    
    return video_path, lip_paths

In [11]:
def collect_examples_for_word(word, num_examples=examples_per_word):
    """
    Collecte plusieurs exemples vidéo pour un mot donné.
    """
    print(f"\n=== Collecte d'exemples pour le mot '{word}' ===\n")
    
    video_paths = []
    all_lip_paths = []
    
    for i in range(1, num_examples + 1):
        print(f"\nExemple {i}/{num_examples} pour '{word}'")
        
        # Enregistrement
        frames = record_video()
        
        # Visualisation
        lip_regions = visualize_recording(frames)
        
        # Demander à l'utilisateur si l'enregistrement est bon
        valid = input("Cet enregistrement est-il bon ? (o/n) : ").lower()
        
        if valid == 'o':
            # Sauvegarde
            video_path, lip_paths = save_video_and_lips(frames, word, i)
            video_paths.append(video_path)
            all_lip_paths.extend(lip_paths)
            print(f"Exemple {i} sauvegardé")
        else:
            print("Enregistrement ignoré. Réessayons.")
            i -= 1  # Réessayer
    
    print(f"\nCollecte terminée pour le mot '{word}' ! {len(video_paths)} exemples sauvegardés.")
    return video_paths, all_lip_paths

In [ ]:
# Pour collecter des exemples pour tous les mots
all_video_paths = {}
all_lip_paths = {}

for word in target_words:
    video_paths, lip_paths = collect_examples_for_word(word)
    all_video_paths[word] = video_paths
    all_lip_paths[word] = lip_paths
    
print("\nCollecte terminée pour tous les mots !")

In [ ]:
def check_data_collection():
    """
    Vérifie les données collectées et affiche un résumé.
    """
    print("=== Résumé des données collectées ===\n")
    
    total_videos = 0
    total_lip_frames = 0
    
    for word in target_words:
        word_video_dir = os.path.join(video_dir, word)
        word_lip_dir = os.path.join(lip_dir, word)
        
        videos = [f for f in os.listdir(word_video_dir) if f.endswith('.mp4')]
        lip_frames = [f for f in os.listdir(word_lip_dir) if f.endswith('.png')]
        
        print(f"Mot '{word}' : {len(videos)} vidéos, {len(lip_frames)} images de lèvres")
        
        total_videos += len(videos)
        total_lip_frames += len(lip_frames)
        
        # Afficher quelques statistiques sur les images de lèvres
        if lip_frames:
            lip_sizes = []
            for lip_file in lip_frames[:5]:  # Limite à 5 fichiers pour économiser du temps
                lip_path = os.path.join(word_lip_dir, lip_file)
                lip_img = cv2.imread(lip_path)
                lip_sizes.append((lip_img.shape[0], lip_img.shape[1]))
            
            if lip_sizes:
                avg_height = np.mean([s[0] for s in lip_sizes])
                avg_width = np.mean([s[1] for s in lip_sizes])
                print(f"  Taille moyenne des images de lèvres : {avg_height:.1f} x {avg_width:.1f} pixels")
    
    print(f"\nTotal : {total_videos} vidéos, {total_lip_frames} images de lèvres")
    
    return total_videos, total_lip_frames

In [ ]:
check_data_collection()
